In [1]:
loans = read.csv("./data/loans.csv")

In [2]:
table(loans$not.fully.paid)


   0    1 
8045 1533 

In [3]:
1533/(8045+1533)

[1] 0.1600543

In [4]:
summary(loans)

 credit.policy                 purpose        int.rate       installment    
 Min.   :0.000   all_other         :2331   Min.   :0.0600   Min.   : 15.67  
 1st Qu.:1.000   credit_card       :1262   1st Qu.:0.1039   1st Qu.:163.77  
 Median :1.000   debt_consolidation:3957   Median :0.1221   Median :268.95  
 Mean   :0.805   educational       : 343   Mean   :0.1226   Mean   :319.09  
 3rd Qu.:1.000   home_improvement  : 629   3rd Qu.:0.1407   3rd Qu.:432.76  
 Max.   :1.000   major_purchase    : 437   Max.   :0.2164   Max.   :940.14  
                 small_business    : 619                                    
 log.annual.inc        dti              fico       days.with.cr.line
 Min.   : 7.548   Min.   : 0.000   Min.   :612.0   Min.   :  179    
 1st Qu.:10.558   1st Qu.: 7.213   1st Qu.:682.0   1st Qu.: 2820    
 Median :10.928   Median :12.665   Median :707.0   Median : 4140    
 Mean   :10.932   Mean   :12.607   Mean   :710.8   Mean   : 4562    
 3rd Qu.:11.290   3rd Qu.:17.950   3rd 

In [2]:
library(mice)
set.seed(144)
vars.for.imputation = setdiff(names(loans), "not.fully.paid")
imputed = complete(mice(loans[vars.for.imputation]))
loans[vars.for.imputation] = imputed

Loading required package: Rcpp
mice 2.25 2015-11-09



 iter imp variable
  1   1  log.annual.inc  days.with.cr.line  revol.util  inq.last.6mths  delinq.2yrs  pub.rec
  1   2  log.annual.inc  days.with.cr.line  revol.util  inq.last.6mths  delinq.2yrs  pub.rec
  1   3  log.annual.inc  days.with.cr.line  revol.util  inq.last.6mths  delinq.2yrs  pub.rec
  1   4  log.annual.inc  days.with.cr.line  revol.util  inq.last.6mths  delinq.2yrs  pub.rec
  1   5  log.annual.inc  days.with.cr.line  revol.util  inq.last.6mths  delinq.2yrs  pub.rec
  2   1  log.annual.inc  days.with.cr.line  revol.util  inq.last.6mths  delinq.2yrs  pub.rec
  2   2  log.annual.inc  days.with.cr.line  revol.util  inq.last.6mths  delinq.2yrs  pub.rec
  2   3  log.annual.inc  days.with.cr.line  revol.util  inq.last.6mths  delinq.2yrs  pub.rec
  2   4  log.annual.inc  days.with.cr.line  revol.util  inq.last.6mths  delinq.2yrs  pub.rec
  2   5  log.annual.inc  days.with.cr.line  revol.util  inq.last.6mths  delinq.2yrs  pub.rec
  3   1  log.annual.inc  days.with.cr.line  revol.

In [3]:
library("caTools")

In [4]:
set.seed(144)
split = sample.split(loans$not.fully.paid, SplitRatio=0.7)
train = subset(loans, split==TRUE)
test = subset(loans, split=FALSE)

In [5]:
loans$purpose = as.factor(loans$purpose)

In [6]:
model1 = glm(not.fully.paid ~ ., data=train, family=binomial)
summary(model1)


Call:
glm(formula = not.fully.paid ~ ., family = binomial, data = train)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.2097  -0.6214  -0.4950  -0.3601   2.6414  

Coefficients:
                    Estimate Std. Error z value Pr(>|z|)    
(Intercept)        9.260e+00  1.554e+00   5.958 2.55e-09 ***
credit.policy     -3.327e-01  1.011e-01  -3.292 0.000995 ***
purpose2          -6.100e-01  1.344e-01  -4.538 5.67e-06 ***
purpose3          -3.181e-01  9.185e-02  -3.463 0.000534 ***
purpose4           1.386e-01  1.753e-01   0.791 0.429074    
purpose5           1.774e-01  1.479e-01   1.199 0.230496    
purpose6          -4.783e-01  2.009e-01  -2.381 0.017260 *  
purpose7           4.159e-01  1.419e-01   2.932 0.003373 ** 
int.rate           6.202e-01  2.085e+00   0.297 0.766111    
installment        1.279e-03  2.093e-04   6.110 9.96e-10 ***
log.annual.inc    -4.357e-01  7.151e-02  -6.093 1.11e-09 ***
dti                4.733e-03  5.501e-03   0.861 0.389508    
fico

In [7]:
predicted.risk = predict(model1, type="response", newdata=test)
table(test$not.fully.paid, predicted.risk>=0.5)

Warning message:
: contrasts dropped from factor purpose

   
    FALSE TRUE
  0  8002   43
  1  1498   35

In [8]:
(8002+35)/(8002+43+1498+35)

[1] 0.8391105

In [9]:
table(test$not.fully.paid)


   0    1 
8045 1533 

In [9]:
8045/(8045+1533)

[1] 0.8399457

In [12]:
library("ROCR")

Loading required package: gplots

Attaching package: 'gplots'

The following object is masked from 'package:stats':

    lowess



In [13]:
ROCRpred = prediction(predicted.risk, test$not.fully.paid)
auc = as.numeric(performance(ROCRpred, "auc")@y.values)
auc

[1] 0.6817063

In [14]:
model2 = glm(not.fully.paid ~ int.rate, data=train, family=binomial)
predTest2 = predict(model2, type="response", newdata=test)

In [15]:
summary(predTest2)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
0.06196 0.11730 0.15080 0.15990 0.19270 0.44340 

In [16]:
table(test$not.fully.paid, predTest2>=0.5)

   
    FALSE
  0  8045
  1  1533

In [17]:
ROCRpred2 = prediction(predTest2, test$not.fully.paid)
auc = as.numeric(performance(ROCRpred2, "auc")@y.values)
auc

[1] 0.6202288

In [17]:
exp(3*0.06)*10

[1] 11.97217

In [18]:
test$profit = exp(test$int.rate*3) - 1
test$profit[test$not.fully.paid == 1] = -1

In [19]:
10 * max(test$profit)

[1] 8.894769

In [19]:
highInterest = subset(test, int.rate>=0.15)
mean(highInterest$profit)

[1] 0.2330475

In [21]:
table(highInterest$not.fully.paid)


   0    1 
1064  350 

In [22]:
350/(1064+350)

[1] 0.2475248

In [10]:
test$predicted.risk = predicted.risk

In [20]:
cutoff = sort(highInterest$predicted.risk, decreasing=FALSE)[100]

In [21]:
cutoff

[1] 0.1367793

In [22]:
selectedLoans = subset(highInterest, predicted.risk<=cutoff)
sum(selectedLoans$profit)

[1] 38.88035

In [48]:
table(selectedLoans$not.fully.paid)


 0  1 
86 14 